In [1]:
import os
os.chdir("../")

In [2]:
import tensorflow as tf
import numpy as np

from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

In [3]:
%load_ext autoreload
%autoreload 2

from diffgp_svi.likelihoods import Gaussian
from diffgp_svi.models import DiffGP
from diffgp_svi.layers import SVGP_Layer
from diffgp_svi.kernels import RBF
from diffgp_svi.datasets import Datasets
from diffgp_svi.settings import Settings

float_type = Settings().tf_float_type

**01. Load dataset **

In [4]:
datasets = Datasets(data_path='data/')
data = datasets.all_datasets['kin8nm'].get_data(split=1,prop=0.9)
Xtrain, Ytrain, Xtest, Ytest, Y_std = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys','Y_std']]

**02. Model and training settings**

In [5]:
num_iter      = 3000 
num_minibatch = min(10000,Xtrain.shape[0])
num_samples   = 4                # Number of MC samples of SDE solutions
num_data      = Xtrain.shape[0]
num_dim       = Xtrain.shape[1]  # Regression input dimensions
num_output    = 1                # Regression output dimensions
num_ind       = 100              # Number of inducing variable in each layer 
flow_time     = 5.0              # SDE integration time
flow_nsteps   = 20               # Number of discretizations in Euler Maruyama solver
white         = True             # Whitened represetation for both differential and predictor GPs 
q_diag        = False            # Diagonal posterior assumption for inducing variables. 
full_cov      = False            # NOTE: full_cov = True is not implemented  yet. 

** 03. Session definition**

In [6]:
# start an interactive session and define separate handles for model training and testing
config = tf.ConfigProto(allow_soft_placement = True,log_device_placement=False)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
sess = tf.InteractiveSession(config = config)

X_placeholder = tf.placeholder(dtype = float_type,shape=[None,None])
Y_placeholder = tf.placeholder(dtype = float_type,shape=[None,1])

train_dataset  = tf.data.Dataset.from_tensor_slices((X_placeholder,Y_placeholder))
train_dataset  = train_dataset.shuffle(buffer_size=num_minibatch*5).repeat()
train_dataset  = train_dataset.batch(num_minibatch).prefetch(buffer_size = num_minibatch)
train_iterator = train_dataset.make_initializable_iterator()

test_dataset  = tf.data.Dataset.from_tensor_slices((X_placeholder,Y_placeholder))
test_dataset  = test_dataset.batch(min(Xtest.shape[0],num_minibatch)).repeat().prefetch(buffer_size = min(Xtest.shape[0],num_minibatch))
test_iterator = test_dataset.make_initializable_iterator()

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types, train_dataset.output_shapes)
X,Y = iterator.get_next()

** 04. Model definition**

In [7]:
# diff GP kernel initialization
# here we use a single RBF kernel with ARD in the differential GP
# this implementation uses same kernel across all dimensions of SDE
# however, in the paper, we use different kernel across dimensions
diff_layer_kernel = RBF(input_dim = num_dim,
                        init_lengthscales = 1.0*num_dim, 
                        init_variance    = 0.01)



# pred GP kernel initialization
pred_layer_kernel = RBF(input_dim = num_dim,
                        init_lengthscales = 1.0*num_dim,
                        init_variance    = 1.0)

# diff GP layer definition
diff_layer = SVGP_Layer(kern    = diff_layer_kernel,
                        Um      = np.random.randn(num_ind,num_dim), 
                        Us_sqrt = np.ones((num_dim,num_ind)) if q_diag else np.stack([np.eye(num_ind)]*num_dim), 
                        Z       = kmeans2(Xtrain,num_ind,minit='points')[0],
                        num_outputs = num_dim,
                        white       = white)

# pred GP layer definition
pred_layer = SVGP_Layer(kern    = pred_layer_kernel,
                        Um      = np.random.randn(num_ind,num_output),
                        Us_sqrt = np.ones((num_output,num_ind)) if q_diag else np.stack([np.eye(num_ind)]*num_output),
                        Z       = kmeans2(Xtrain,num_ind,minit='points')[0],
                        num_outputs  = num_output,
                        white        = white)

# model definition
model = DiffGP(likelihood  = Gaussian(), 
               diff_layer  = diff_layer,
               pred_layer  = pred_layer,
               num_samples = num_samples,
               flow_time   = flow_time,
               flow_nsteps = flow_nsteps,
               num_data    = num_data)

** 05. Optimization objective and summary statistics**

In [8]:
with tf.device('/gpu:0'):
    # Compute model lowerbound (averaged over samples)
    lowerbound = model._build_likelihood(X,Y)

    # Minimize negative lowerbound
    train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(-1*lowerbound)
    
    # Compute predictive distributions and then generate training summary (log likelihood and RMSE)
    predictive_mean, predictive_var = model.predict_y(X,num_samples=5)
    predictive_dist = tf.distributions.Normal(loc=predictive_mean*Y_std, scale=Y_std*predictive_var**0.5)
    summ_loglik     = predictive_dist.log_prob(Y*Y_std)
    summ_loglik     = tf.reduce_mean(tf.reduce_logsumexp(summ_loglik - tf.log(tf.cast(tf.shape(predictive_mean)[0],dtype=float_type)), axis=0))
    summ_rmse       = tf.sqrt(tf.reduce_mean(Y_std**2*((tf.reduce_mean(predictive_mean,axis=0) - Y)**2)))

In [9]:
# tensorflow variable and handle initializations
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

train_handle  = sess.run(train_iterator.string_handle())
sess.run(train_iterator.initializer,{X_placeholder:Xtrain,Y_placeholder:Ytrain})

test_handle = sess.run(test_iterator.string_handle())
sess.run(test_iterator.initializer,{X_placeholder:Xtest,Y_placeholder:Ytest})

** 05. Model training **

In [10]:
print('{:>5s}'.format("iter")        + '{:>12s}'.format("lowerbound")+ \
      '{:>12s}'.format("train_rmse") + '{:>12s}'.format("test_rmse")+\
      '{:>12s}'.format("train_ll")   + '{:>12s}'.format("test_ll"))

for i in range(1,num_iter+1):
    
    try:
        sess.run(train_op,feed_dict={handle:train_handle})
        
        # print every 100 iterations 
        if i % 50 == 0:
            
            # lowerbound, train rmse, train log lik
            _lb,_trn_rmse,_trn_ll = sess.run([lowerbound,summ_rmse,summ_loglik],
                                             {handle:train_handle})
            # test rmse, test log lik
            _tst_rmse,_tst_ll = sess.run([summ_rmse,summ_loglik],
                                         {handle:test_handle})
            
            print('{:>5d}'.format(i)            + '{:>12.4f}'.format(_lb)+\
                  '{:>12.4f}'.format(_trn_rmse) + '{:>12.4f}'.format(_tst_rmse)+\
                  '{:>12.4f}'.format(_trn_ll)   + '{:>12.4f}'.format(_tst_ll))

    except KeyboardInterrupt as e:
        print("stopping training")
        break

 iter  lowerbound  train_rmse   test_rmse    train_ll     test_ll
   50     -1.2733      0.1937      0.1894      0.1036      0.1133
  100     -1.1328      0.1752      0.1753      0.2282      0.2280
  150     -0.9429      0.1382      0.1399      0.4306      0.4262
  200     -0.6774      0.0935      0.0979      0.7204      0.7059
  250     -0.4702      0.0795      0.0836      0.9492      0.9301
  300     -0.3378      0.0726      0.0764      1.1146      1.0886
  350     -0.2768      0.0696      0.0732      1.2060      1.1730
  400     -0.2480      0.0674      0.0721      1.2554      1.2044
  450     -0.2365      0.0666      0.0699      1.2741      1.2378
  500     -0.2281      0.0656      0.0697      1.2901      1.2472
  550     -0.2261      0.0651      0.0704      1.2987      1.2382
  600     -0.2213      0.0645      0.0683      1.3082      1.2660
  650     -0.2155      0.0642      0.0685      1.3130      1.2598
  700     -0.2105      0.0639      0.0683      1.3188      1.2661
  750     

** 06. Results on test set **

In [11]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(X.shape[0]/num_minibatch), 1)
    lik, sq_diff = [], []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        l, sq = assess_model(model, X_batch, Y_batch)
        lik.append(l)
        sq_diff.append(sq)
    lik = np.concatenate(lik, 0)
    sq_diff = np.array(np.concatenate(sq_diff, 0), dtype=float)
    return np.average(lik), np.average(sq_diff)**0.5

S = 50
def assess_sampled(model, X_batch, Y_batch):
    m, v = model.predict_y(X, S)
    m = m.eval({X:X_batch});v = v.eval({X:X_batch});
    S_lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5), 2)
    lik = logsumexp(S_lik, (0),b=1/(float(m.shape[0])))
    
    mean = np.average(m,0)
    sq_diff = Y_std**2*((mean - Y_batch)**2)

    return lik, sq_diff

In [12]:
lik, rmse = batch_assess(model, assess_sampled, Xtest, Ytest)
print("final loglik: "+str(np.round(lik,4)))
print("final rmse  : "+str(np.round(rmse,4)))

final loglik: 1.3137
final rmse  : 0.0652
